<a href="https://colab.research.google.com/github/ukenia/quantization-cnn/blob/starter-code/Wav2Letter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Apr  7 22:17:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
BASE_PATH = "/content/gdrive/MyDrive/idl_proj/"
DATA_PATH = BASE_PATH + "data/"
MODEL_PATH = BASE_PATH + "models/"
PREDICTION_PATH = BASE_PATH + "predictions/"

In [13]:
# Files from drive
train_filename = DATA_PATH + "train.npy"
train_transcripts_filename = DATA_PATH + "train_transcripts.npy"

dev_filename = DATA_PATH + "dev.npy"
dev_transcripts_filename = DATA_PATH + "dev_transcripts.npy"

test_filename = DATA_PATH + "test.npy"

In [5]:
# Installing CTC Decoder
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1063, done.
remote: Total 1063 (delta 0), reused 0 (delta 0), pack-reused 1063
Receiving objects: 100% (1063/1063), 759.71 KiB | 18.99 MiB/s, done.
Resolving deltas: 100% (513/513), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 13687, done.        
remote: Total 13687 (delta 0), reused 0 (delta 0), pack-reused 13687        
Receiving objects: 100% (13687/13687), 5.46 MiB | 22.82 MiB/s, done.
Resolving deltas: 100% (7880/7880), done.
Submodule path 'third_party/ThreadPool': checked out '

In [6]:
# Code starts here!

In [7]:
# Import necessary libraries

import timeit
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.nn.utils.rnn import *

from ctcdecode import CTCBeamDecoder
from datetime import datetime as dt

In [8]:
# Check if cuda is available

cuda = torch.cuda.is_available()
num_workers = 4 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()

Cuda = True with num_workers = 4


In [9]:
class Wav2LetterDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path, allow_pickle=True)
        self.y = np.load(y_path, allow_pickle=True)

        self.length = self.X.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]

        return x, y

def pad_collate(batch):
    data = [torch.LongTensor(item[0]) for item in batch]
    data_lengths = torch.LongTensor([len(seq) for seq in data])
    data = pad_sequence(data)
  
    target = [torch.LongTensor(item[1]) for item in batch]
    target_lengths = torch.LongTensor([len(seq) for seq in target])
    target = pad_sequence(target, batch_first=True)
  
    return data, target, data_lengths, target_lengths

In [14]:
hyperparameters = {
    "batch_size": 32,
    "epochs": 50,
    "learning_rate": 5e-3,
    "weight_decay": 1e-5,
}

In [16]:
train_data = Wav2LetterDataset(train_filename, train_transcripts_filename)
train_args = dict(shuffle=True, batch_size=hyperparameters["batch_size"], num_workers=num_workers, drop_last=True, collate_fn=pad_collate)
train_loader = DataLoader(train_data, **train_args)

val_data = Wav2LetterDataset(dev_filename, dev_transcripts_filename)
val_args = dict(shuffle=True, batch_size=hyperparameters["batch_size"], num_workers=num_workers, drop_last=True, collate_fn=pad_collate)
val_loader = DataLoader(val_data, **val_args)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv(out)
        out = self.relu(out)

        return out

In [29]:
class Wav2Letter(nn.Module):

    def __init__(self, num_classes = 42, num_features = 40):
        super(Wav2Letter, self).__init__()

        model = nn.Sequential(
            ConvBlock(in_channels=num_features, out_channels=250, kernel_size=48, stride=2, padding=23),

            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),
            ConvBlock(in_channels=250, out_channels=250, kernel_size=7, stride=1, padding=3),

            ConvBlock(in_channels=250, out_channels=2000, kernel_size=32, stride=1, padding=16),
            ConvBlock(in_channels=2000, out_channels=2000, kernel_size=1, stride=1, padding=0),
            ConvBlock(in_channels=2000, out_channels=num_classes, kernel_size=1, stride=1, padding=0)
        )
        
        self.model = model
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        # Input - (batch_size, num_features, input_length)
        out = self.model(x)
        out = self.log_softmax(out)
        return out

In [30]:
model = Wav2Letter()
model.to(device)

Wav2Letter(
  (model): Sequential(
    (0): ConvBlock(
      (conv): Conv1d(40, 250, kernel_size=(48,), stride=(2,), padding=(23,))
      (relu): ReLU(inplace=True)
    )
    (1): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (2): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (3): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (4): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (5): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (6): ConvBlock(
      (conv): Conv1d(250, 250, kernel_size=(7,), stride=(1,), padding=(3,))
      (relu): ReLU(inplace=True)
    )
    (7): ConvBlock(

In [31]:
criterion = nn.CTCLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters["learning_rate"], weight_decay=hyperparameters["weight_decay"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5) # OR can use some other scheduler

In [32]:
# Train the model - Change based on the model

def train_model(train_loader, model):
    training_loss = 0
    
    # Set model in 'Training mode'
    model.train()
    
    # enumerate mini batches
    for i, (inputs, targets, input_lengths, target_lengths) in enumerate(train_loader):

        inputs = inputs.transpose(0,1).transpose(1,2).reshape((batch_size, 1, 40, -1)).to(device)        
        targets = targets.to(device)
        
        # clear the gradients
        optimizer.zero_grad()
        
        # compute the model output
        out = model(inputs.float())
        
        # calculate loss
        loss = criterion(out, targets)
        
        # Backward pass
        loss.backward()
        
        # Update model weights
        optimizer.step()

        training_loss += loss.item()
    training_loss /= len(train_loader)
    return training_loss

In [33]:
# Evaluate the model - Change based on the model

def evaluate_model(val_loader, model):
    
    model.eval()
    
    # enumerate mini batches
    for i, (inputs, targets, input_lengths, target_lengths) in enumerate(val_loader):

        inputs = inputs.transpose(0,1).transpose(1,2).reshape((batch_size, 1, 40, -1)).to(device)        
        targets = targets.to(device)
        
        # compute the model output
        out = model(inputs.float())
        
        # calculate loss
        loss = criterion(out, targets)

        decoder = CTCBeamDecoder(label_map, beam_width=4, log_probs_input=True)
        out, _, _, out_lengths = decoder.decode(out.transpose(0, 1), out_lengths)
        

    return loss.item()

In [35]:
for epoch in range(hyperparameters["epochs"]):
    print("Epoch: ", epoch)

    if epoch % 10 == 0 and epoch != 0:
      torch.save(model, MODEL_PATH + "base_model.pth")

    # Train
    starttime = timeit.default_timer()
    training_loss = train_model(train_loader, model)
    endtime = timeit.default_timer()
    print("Training time: ", (endtime - starttime)/60)

    # Validation
    starttime = timeit.default_timer()
    val_dist, val_loss = evaluate_model(val_loader, model)
    endtime = timeit.default_timer()
    print("Validation time: ", (endtime - starttime)/60)

    scheduler.step(val_loss)

    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss)+", Validation loss: "+str(val_loss)+
          ", Validation distance: "+str(val_dist)+", LR: "+str(scheduler.get_last_lr())+"\n")

Epoch:  0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored